# High-level MXNet Example

**In the interest of comparison; a common (custom) data-generator (called yield_mb(X, y, batchsize=64, shuffle=False)) was originally used for all other frameworks - but not for MXNet. I have reproduced the MXNet example using this same generator (wrapping the results in the mx.io.DataBatch class) to test if MXNet is faster than other frameworks just because I was using its own data-generator. This does not appear to be the case. **

In [1]:
# Parameters
EPOCHS=5
N_CLASSES=10
BATCHSIZE=64
LR=0.01
MOMENTUM=0.9
GPU=True

In [2]:
import os
from os import path
import sys
import numpy as np
import mxnet as mx
from utils import cifar_for_library, yield_mb
from nb_logging import NotebookLogger, output_to, error_to

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("MXNet: ", mx.__version__)

('OS: ', 'linux2')
('Python: ', '2.7.12 (default, Nov 19 2016, 06:48:10) \n[GCC 5.4.0 20160609]')
('Numpy: ', '1.11.0')
('MXNet: ', '0.11.0')


In [4]:
nb_teminal_logger = NotebookLogger(sys.stdout.session, sys.stdout.pub_thread, sys.stdout.name, sys.__stdout__)

In [5]:
rst_out = output_to(nb_teminal_logger)
rst_err = error_to(nb_teminal_logger)

In [6]:
data_path = path.join(os.getenv('AZ_LEARNING_INPUT_DATASET'), 'cifar-10-batches-py')

In [7]:
def create_symbol():
    data = mx.symbol.Variable('data')
    # size = [(old-size - kernel + 2*padding)/stride]+1
    # if kernel = 3, pad with 1 either side
    conv1 = mx.symbol.Convolution(data=data, num_filter=50, pad=(1,1), kernel=(3,3))
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    conv2 = mx.symbol.Convolution(data=relu1, num_filter=50, pad=(1,1), kernel=(3,3))
    relu2 = mx.symbol.Activation(data=conv2, act_type="relu")
    pool1 = mx.symbol.Pooling(data=relu2, pool_type="max", kernel=(2,2), stride=(2,2))
    drop1 = mx.symbol.Dropout(data=pool1, p=0.25)
    
    conv3 = mx.symbol.Convolution(data=drop1, num_filter=100, pad=(1,1), kernel=(3,3))
    relu3 = mx.symbol.Activation(data=conv3, act_type="relu")
    conv4 = mx.symbol.Convolution(data=relu3, num_filter=100, pad=(1,1), kernel=(3,3))
    relu4 = mx.symbol.Activation(data=conv4, act_type="relu")
    pool2 = mx.symbol.Pooling(data=relu4, pool_type="max", kernel=(2,2), stride=(2,2))
    drop2 = mx.symbol.Dropout(data=pool2, p=0.25)
           
    flat1 = mx.symbol.Flatten(data=drop2)
    fc1 = mx.symbol.FullyConnected(data=flat1, num_hidden=512)
    relu7 = mx.symbol.Activation(data=fc1, act_type="relu")
    drop4 = mx.symbol.Dropout(data=relu7, p=0.5)
    fc2 = mx.symbol.FullyConnected(data=drop4, num_hidden=N_CLASSES) 
    
    input_y = mx.symbol.Variable('softmax_label')  
    m = mx.symbol.SoftmaxOutput(data=fc2, label=input_y, name="softmax")
    return m

In [8]:
def init_model(m):
    if GPU:
        ctx = [mx.gpu(0)]
    else:
        ctx = mx.cpu()
    
    mod = mx.mod.Module(context=ctx, symbol=m)
    mod.bind(data_shapes=[('data', (BATCHSIZE, 3, 32, 32))],
             label_shapes=[('softmax_label', (BATCHSIZE,))])

    # Glorot-uniform initializer
    mod.init_params(initializer=mx.init.Xavier(rnd_type='uniform'))
    mod.init_optimizer(optimizer='sgd', 
                       optimizer_params=(('learning_rate', LR), ('momentum', MOMENTUM), ))
    return mod

In [9]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(data_path, channel_first=True)

# Load data-iterator
#train_iter = mx.io.NDArrayIter(x_train, y_train, BATCHSIZE, shuffle=True)
# Use custom iterator instead of mx.io.NDArrayIter() for consistency
# Wrap as DataBatch class
wrapper_db = lambda args: mx.io.DataBatch(data=[mx.nd.array(args[0])], label=[mx.nd.array(args[1])])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...


Preparing test set...
Done.


((50000, 3, 32, 32), (10000, 3, 32, 32), (50000,), (10000,))
(dtype('float32'), dtype('float32'), dtype('int32'), dtype('int32'))
CPU times: user 828 ms, sys: 584 ms, total: 1.41 s
Wall time: 1.41 s


In [10]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 1.23 ms


In [11]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 2.42 s, sys: 2.48 s, total: 4.9 s
Wall time: 6.04 s


In [12]:
%%time
# Train and log accuracy
metric = mx.metric.create('acc')
for j in range(EPOCHS):
    #train_iter.reset()
    metric.reset()
    #for batch in train_iter:
    for batch in map(wrapper_db, yield_mb(x_train, y_train, BATCHSIZE, shuffle=True)):
        model.forward(batch, is_train=True) 
        model.update_metric(metric, batch.label)
        model.backward()              
        model.update()
    print('Epoch %d, Training %s' % (j, metric.get()))

Epoch 0, Training ('accuracy', 0.3308058578745198)


Epoch 1, Training ('accuracy', 0.5026008322663252)


Epoch 2, Training ('accuracy', 0.5910091229193342)


Epoch 3, Training ('accuracy', 0.6461867797695262)


Epoch 4, Training ('accuracy', 0.6851392445582587)
CPU times: user 34.7 s, sys: 27.8 s, total: 1min 2s
Wall time: 56.5 s


In [13]:
%%time
y_guess = model.predict(mx.io.NDArrayIter(x_test, batch_size=BATCHSIZE, shuffle=False))
y_guess = np.argmax(y_guess.asnumpy(), axis=-1)

CPU times: user 652 ms, sys: 432 ms, total: 1.08 s
Wall time: 1.16 s


In [14]:
print("Accuracy: ", float(sum(y_guess == y_test))/len(y_guess))

('Accuracy: ', 0.7126)
